In [ ]:
!pip install livelossplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchvision.transforms import ToTensor, Lambda
from tqdm import tqdm
from livelossplot import PlotLosses

In [ ]:
torch.manual_seed(42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tesla T4
Memory Usage:
Allocated: 0.1 GB
Cached:    7.8 GB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/COVID-19_Radiography_Dataset.zip', 'r') as f:
  f.extractall()

In [ ]:
label_map ={
    0 : "Normal",
    1 : "Covid",
    2 : "Lung_Opacity",
    3 : "Viral Pneumonia"
}

In [ ]:
masks_pwd = ["COVID-19_Radiography_Dataset/Normal/masks/*","COVID-19_Radiography_Dataset/COVID/masks/*", 
"COVID-19_Radiography_Dataset/Lung_Opacity/masks/*","COVID-19_Radiography_Dataset/Viral Pneumonia/masks/*"]
images_pwd = ["COVID-19_Radiography_Dataset/Normal/images/*","COVID-19_Radiography_Dataset/COVID/images/*", 
"COVID-19_Radiography_Dataset/Lung_Opacity/images/*","COVID-19_Radiography_Dataset/Viral Pneumonia/images/*"]
image_paths = []
labels = []
mask_paths = []
for i, (folder_img_path, folder_mask_path) in enumerate(zip(images_pwd, masks_pwd)):
    for f_path, m_path in zip(glob(folder_img_path), glob(folder_mask_path)):
        image_paths.append(f_path)
        mask_paths.append(m_path)
        labels.append(i)

print("Images: ", len(image_paths), ", Labels: ", len(labels), ", Masks: " ,len(mask_paths))

Images:  21165 , Labels:  21165 , Masks:  21165


In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
class PulmonesDataset:
    def __init__(self, img_paths, img_labels):
        self.img_paths = img_paths
        self.img_labels = img_labels
    
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        PULMON_IMAGE = Image.open(self.img_paths[index]).convert('L') # Convierte gray-scale
        TENSOR_IMAGE = transform(PULMON_IMAGE)
        label = self.img_labels[index]
        return TENSOR_IMAGE, label

In [ ]:
pulmon = Image.open("COVID-19_Radiography_Dataset/Normal/images/Normal-1.png").convert('L')

In [ ]:
dataset = PulmonesDataset(image_paths, labels)

In [ ]:
dataset_indices = list(range(0, len(dataset)))
dataset_labels = labels
train_indices, test_indices = train_test_split(dataset_indices, test_size=0.3, random_state = 1, stratify= dataset_labels )

test_labels = [dataset_labels[idx] for idx in test_indices]
validate_indices, test_indices = train_test_split(test_indices, test_size = 0.33, random_state = 1, stratify= test_labels)
print(f"Number of training examples: {len(train_indices)}")
print(f"Number of validation examples: {len(validate_indices)}")
print(f"Number of test examples: {len(test_indices)}")

Number of training examples: 14815
Number of validation examples: 4254
Number of test examples: 2096


In [ ]:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validate_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [ ]:
print(len(train_sampler))
print(len(validation_sampler))
print(len(test_sampler))

14815
4254
2096


# Preparing Data Loader Objects

In [1]:
BATCH_SIZE = 8
train_loader = DataLoader(dataset, batch_size= BATCH_SIZE, sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=validation_sampler)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler= test_sampler)

NameError: ignored

In [ ]:
print(len(train_loader))
print(len(validation_loader))
print(len(test_loader))

In [ ]:
images, labels = next(iter(train_loader))
type(labels)

In [ ]:
images, labels = next(iter(train_loader))
fig, axis= plt.subplots(4, 6, figsize=(15, 18))
for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        npimg = images[i].numpy()
        npimg = np.transpose(npimg, (1, 2, 0))
        npimg = np.reshape(npimg, (299,299))
        label = label_map[int(labels[i])]
        ax.imshow(npimg)
        ax.set(title=f"{label}")
        

In [ ]:
def get_composition(index_lst: list, test_name = "generic"):
    counter = {}
    for label in label_map:
        counter[label_map[label]] = 0

    for inx in index_lst:
        label = dataset_labels[inx]
        counter[label_map[label] ] += 1

    return pd.DataFrame.from_dict(counter, orient= 'index', columns= [test_name]) / len(index_lst)


distSet = [ (train_indices, "train"), (validate_indices,"validate") , (test_indices,"test")]
df_lst = [ get_composition(idx, name) for idx, name in distSet]
df_dst = pd.concat( df_lst, axis = 1) 

df_dst 

In [ ]:
num_classes = 4
learning_rate = 0.001
num_epochs = 20

In [ ]:
from torchvision import transforms, models

In [ ]:
from collections import OrderedDict


In [ ]:
class CNN5(nn.Module):
    def __init__(self, num_classes = 4):
        super(CNN5, self).__init__()
        self.flatten = nn.Flatten() # Reduce cualquier matriz a 1D
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),# 256 -> 258 (mask) 299 -> 301
            nn.Dropout(p=0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 129 | 150
            nn.BatchNorm2d(16)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), # 129 -> 129 (mask) | 150-> 150
            nn.Dropout(p=0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 129 // 2 = 64 (mask) | 75
            nn.BatchNorm2d(32)

        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size= 3, stride= 1, padding=1), # 64 -> 64 | 75 -> 75
            nn.Dropout(p=0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 64 // 2 -> 32 || 75 // 2 -> 37
            nn.BatchNorm2d(64)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size= 3, stride=1, padding=1), # 32 -> 32
            nn.Dropout(p=0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 16 -> 16
            nn.BatchNorm2d(128)
        )
        self.fc = nn.Linear(18*18*128, 128 ) 
        self.act = nn.Sigmoid()
        self.fc2 = nn.Linear(128, num_classes) 

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.act(out)
        out = self.fc2(out)
        return out

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes = 4):
        super(CNN, self).__init__()
        self.den = models.densenet121(pretrained=False)
        self.features = nn.Sequential(
    nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1, bias=False)),
            ('norm0', nn.BatchNorm2d(64)),
            ('relu0', nn.ReLU(inplace=True))
        ])), self.den.features[4:])
        self.classifier = nn.Sequential(
    nn.Linear(1024, 512),
    nn.Dropout(p=0.1),
    nn.ReLU(),
    nn.Linear(512, 4)
)

    def forward(self, x):
        out = self.den(x)
        out = self.features(out)
        out = out.reshape(out.size(0), -1)
        out = self.classifier(out)
        return out


In [ ]:
net = models.densenet121(pretrained=False)
net.features = nn.Sequential(
    nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(1, 64, kernel_size=3, stride=2, padding=1, bias=False)),
            ('norm0', nn.BatchNorm2d(64)),
            ('relu0', nn.ReLU(inplace=True))
        ])),  # 96**2 -> 48**2
    net.features[4:])

net.classifier = nn.Sequential(
    nn.Linear(1024, 512),
    nn.Dropout(p=0.1),
    nn.ReLU(),
    nn.Linear(512, 4)
)

In [ ]:
def showLoss(loss, val, mensaje = "Loss", valmsg = "Validation"):
    plt.subplots(figsize=(6, 4))
    plt.plot(range(len(loss)), loss, color="blue", label=mensaje)
    plt.plot(range(len(val)), val, color="orange", label=valmsg)
    plt.legend()
    plt.draw()

In [ ]:
def getAccuracy(model, loader):
    with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
      return 100 * correct / total

In [ ]:
model = CNN5(4).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
def train(model, optimizer, loss_fn, num_epochs, patience = 3):
  # train the model
  liveplot= PlotLosses()
  loss_lst = []
  val_loss_lst = []
  early_stopping = EarlyStopping(patience=patience, verbose=True)

  for epoch in tqdm(range(num_epochs), desc = "Training model..."):
    logs = {}
    epoch_total = 0.
    validation_total = 0.
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)
      # forward 
      output = model(images)
      loss   = loss_fn(output, labels)
      # change the params
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_total += loss.item()
      
    with torch.set_grad_enabled(False):
      for i, (images, labels) in enumerate(validation_loader):
        # Transfer to GPU
        images = images.to(device)
        labels = labels.to(device)
        # forward 
        output = model(images)
        loss   = loss_fn(output, labels)

        validation_total += loss.item()
        
    loss = epoch_total / len(train_loader)
    val_loss = validation_total /  len(validation_loader)
    logs["loss"] , logs["val_loss"] = loss, val_loss
    logs["accuracy"] = getAccuracy(model, train_loader)
    logs["val_accuracy"] = getAccuracy(model, validation_loader)
    loss_lst.append(loss)
    val_loss_lst.append(val_loss)

    liveplot.update(logs)
    liveplot.send()

    early_stopping(val_loss = val_loss, model = model)
    if early_stopping.early_stop:
      print("Early stopping")
      break

  model.load_state_dict(torch.load('checkpoint.pt'))            
  return model, loss_lst, val_loss_lst

In [ ]:

model_result, loss_result, val_loss_result = train(model, optimizer, loss_fn, num_epochs = 20)

In [ ]:
acc_val, acc_test = getAccuracy(model_result, validation_loader) , getAccuracy(model_result, test_loader)
print(f"Validation Accuracy of: {acc_val}", f"Test Accuracy of: {acc_test }")

In [ ]:
def showEarly(train_loss, valid_loss):
    # visualize the loss as the network trained
    fig = plt.figure(figsize=(10,8))
    plt.plot(range(1,len(train_loss)+1),train_loss, label='Training Loss')
    plt.plot(range(1,len(valid_loss)+1),valid_loss,label='Validation Loss')

    # find position of lowest validation loss
    minposs = valid_loss.index(min(valid_loss))+1 
    plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.ylim(0, max(max(train_loss), max(valid_loss)) ) # consistent scale
    plt.xlim(0, len(train_loss)+1) # consistent scale
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
    fig.savefig('loss_plot.png', bbox_inches='tight')

In [ ]:
showEarly(loss_result, val_loss_result)

In [ ]:
print("Validation")
getAccuracy(model, validation_loader)

In [ ]:
print("Test")
getAccuracy(model, test_loader)

# Agregar una capa adicional

In [ ]:
class CNN1(nn.Module):
    def __init__(self, num_classes = 4):
        super(CNN1, self).__init__()
        self.flatten = nn.Flatten() # Reduce cualquier matriz a 1D
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),# 256 -> 258 (mask) 299 -> 301
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # 129 | 150
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), # 129 -> 129 (mask) | 150-> 150
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # 129 // 2 = 64 (mask) | 75
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size= 3, stride= 1, padding=1), # 64 -> 64 | 75 -> 75
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # 64 // 2 -> 32 || 75 // 2 -> 37
        )

        self.fc = nn.Linear(32*32*64, num_classes) # 32 * 32 * 32 (mask) | 37 * 37 * 64

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


In [ ]:
model1 = CNN1(num_classes).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=learning_rate)
print([ e.shape  for e in model1.fc.parameters()])
model1.fc.weight


[torch.Size([4, 65536]), torch.Size([4])]


Parameter containing:
tensor([[ 1.6638e-03,  2.9210e-03, -2.6541e-03,  ..., -6.6661e-04,
         -6.4705e-04,  2.5629e-03],
        [-1.6599e-03, -2.9030e-03, -6.0821e-04,  ..., -9.5855e-04,
         -2.0415e-03, -1.0899e-03],
        [ 3.7865e-03,  3.4328e-03,  2.1990e-03,  ...,  3.8254e-03,
          2.8717e-03, -2.1679e-04],
        [-3.2991e-03,  5.6185e-05, -1.2430e-03,  ..., -3.7531e-03,
         -1.3773e-03, -3.4612e-03]], device='cuda:0', requires_grad=True)

In [ ]:
list_loss = train(model1, optimizer, loss_fn, 20)

Epoch [1/20], Step [100/463], Loss: 1.1090
Epoch [1/20], Step [200/463], Loss: 0.9085
Epoch [1/20], Step [300/463], Loss: 0.7866
Epoch [1/20], Step [400/463], Loss: 0.9533
Epoch [2/20], Step [100/463], Loss: 1.0763
Epoch [2/20], Step [200/463], Loss: 0.9757
Epoch [2/20], Step [300/463], Loss: 0.7258
Epoch [2/20], Step [400/463], Loss: 0.8709
Epoch [3/20], Step [100/463], Loss: 0.4516
Epoch [3/20], Step [200/463], Loss: 0.5758
Epoch [3/20], Step [300/463], Loss: 0.7123
Epoch [3/20], Step [400/463], Loss: 0.5658
Epoch [4/20], Step [100/463], Loss: 0.4859
Epoch [4/20], Step [200/463], Loss: 0.7554
Epoch [4/20], Step [300/463], Loss: 0.8973
Epoch [4/20], Step [400/463], Loss: 0.5337
Epoch [5/20], Step [100/463], Loss: 0.6448
Epoch [5/20], Step [200/463], Loss: 0.5963
Epoch [5/20], Step [300/463], Loss: 0.8689
Epoch [5/20], Step [400/463], Loss: 0.7573
Epoch [6/20], Step [100/463], Loss: 0.5443
Epoch [6/20], Step [200/463], Loss: 0.4414
Epoch [6/20], Step [300/463], Loss: 0.4296
Epoch [6/20

In [ ]:
print("Validation")
getAccuracy(model1, validation_loader)

Validation
Test Accuracy of the model: 65.5618241654913 %


In [ ]:
print("Test")
getAccuracy(model1, test_loader)

Test
Test Accuracy of the model: 64.79007633587786 %


# Agregar dos capas adicionales

In [ ]:
class CNN2(nn.Module):
    def __init__(self, num_classes = 4):
        super(CNN2, self).__init__()
        self.flatten = nn.Flatten() # Reduce cualquier matriz a 1D
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),# 256 -> 258 (mask) 299 -> 301
            nn.Dropout(p = 0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 129 | 150
            nn.BatchNorm2d(16)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), # 129 -> 129 (mask) | 150-> 150
            nn.Dropout(p = 0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 129 // 2 = 64 (mask) | 75
            nn.BatchNorm2d(32)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size= 3, stride= 1, padding=1), # 64 -> 64 | 75 -> 75
            nn.Dropout(p = 0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 64 // 2 -> 32 || 75 // 2 -> 37
            nn.BatchNorm2d(64)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size= 3, stride=1, padding=1), # 32 -> 32
            nn.Dropout(p = 0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 16 -> 16
            nn.BatchNorm2d(128)
        )

        self.fc = nn.Linear(18*18*128, num_classes) # 32 * 32 * 32 (mask) | 18 * 18 * 128

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


## Agregando batch normalization

In [ ]:
torch.manual_seed(50)
class CNN3(nn.Module):
    def __init__(self, num_classes = 4):
        super(CNN3, self).__init__()
        self.flatten = nn.Flatten() # Reduce cualquier matriz a 1D
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),# 256 -> 258 (mask) 299 -> 301
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 129 | 150
            nn.BatchNorm2d(16)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), # 129 -> 129 (mask) | 150-> 150
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),# 129 // 2 = 64 (mask) | 75
            nn.BatchNorm2d(32)
        )
        self.fc = nn.Linear(75*75*32, num_classes) # 64 * 64 * 32 (mask) | 75 * 75 * 32

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


In [ ]:
network3 = CNN3(num_classes=num_classes).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(network3.parameters(), lr=learning_rate)
print([ e.shape  for e in network3.fc.parameters()])
network3.fc.weight

[torch.Size([4, 180000]), torch.Size([4])]


Parameter containing:
tensor([[ 0.0007,  0.0011,  0.0010,  ..., -0.0022, -0.0010,  0.0019],
        [ 0.0012,  0.0002, -0.0013,  ..., -0.0015, -0.0016, -0.0009],
        [ 0.0014,  0.0023,  0.0001,  ...,  0.0020,  0.0009,  0.0014],
        [-0.0018, -0.0005,  0.0022,  ...,  0.0016,  0.0022,  0.0017]],
       device='cuda:0', requires_grad=True)

In [ ]:
list_loss = train(network3, optimizer, loss_fn, 20)

Epoch [1/20], Step [100/463], Loss: 9.1941
Epoch [1/20], Step [200/463], Loss: 2.2015
Epoch [1/20], Step [300/463], Loss: 0.3813
Epoch [1/20], Step [400/463], Loss: 3.4633
Epoch [2/20], Step [100/463], Loss: 3.9101
Epoch [2/20], Step [200/463], Loss: 1.2899
Epoch [2/20], Step [300/463], Loss: 1.6355
Epoch [2/20], Step [400/463], Loss: 1.0140
Epoch [3/20], Step [100/463], Loss: 0.2367
Epoch [3/20], Step [200/463], Loss: 0.2914
Epoch [3/20], Step [300/463], Loss: 0.2021
Epoch [3/20], Step [400/463], Loss: 0.7075
Epoch [4/20], Step [100/463], Loss: 0.0856
Epoch [4/20], Step [200/463], Loss: 0.2142
Epoch [4/20], Step [300/463], Loss: 0.7762
Epoch [4/20], Step [400/463], Loss: 0.2428
Epoch [5/20], Step [100/463], Loss: 0.0271
Epoch [5/20], Step [200/463], Loss: 0.5581
Epoch [5/20], Step [300/463], Loss: 0.0002
Epoch [5/20], Step [400/463], Loss: 1.0190
Epoch [6/20], Step [100/463], Loss: 0.1700
Epoch [6/20], Step [200/463], Loss: 0.0004
Epoch [6/20], Step [300/463], Loss: 0.2338
Epoch [6/20

In [ ]:
print("Validation")
getAccuracy(network3, validation_loader)

Validation
Test Accuracy of the model: 83.23930418429713 %


In [ ]:
print("Test")
getAccuracy(network3, test_loader)

Test
Test Accuracy of the model: 83.30152671755725 %


## Agregando Batch Normalization y Dropout

In [ ]:
torch.manual_seed(50)
class CNN4(nn.Module):
    def __init__(self, num_classes = 4):
        super(CNN3, self).__init__()
        self.flatten = nn.Flatten() # Reduce cualquier matriz a 1D
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),# 256 -> 258 (mask) 299 -> 301
            nn.Dropout(p = 0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 129 | 150
            nn.BatchNorm2d(16)
        )
        # Comentar el dropout y batch normalization para el siguiente experimento
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), # 129 -> 129 (mask) | 150-> 150
            nn.Dropout(p = 0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),# 129 // 2 = 64 (mask) | 75
            nn.BatchNorm2d(32)
        )
        self.fc = nn.Linear(75*75*32, num_classes) # 64 * 64 * 32 (mask) | 75 * 75 * 32

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
